In [1]:
print("Hello")

Hello


# Study pandas dataframe

In [106]:
import pandas as pd

data = {
  "calories": [420, 380, 390],
  "duration": [50, 40, 45],
  "fat" : ["tired","hungry","Energetic"]
}

#load data into a DataFrame object:
df1 = pd.DataFrame(data)

print(df) 

import pandas as pd

data2 = {
    "food" : ["nasi", "bread" , "roti canai"],
    "price" : [10,5,3]
}

df2= pd.DataFrame(data2)

print(df2)

print(df2.loc[2])

   calories  duration
0       420        50
1       380        40
2       390        45
         food  price
0        nasi     10
1       bread      5
2  roti canai      3
food     roti canai
price             3
Name: 2, dtype: object


In [111]:
import pandas as pd

data = {
  "calories": [420, 380, 390],
  "duration": [50, 40, 45],
  "fat" : ["tired","hungry","Energetic"]

}

data2 = {
    "x" : [1,2] ,
    "y" : ['a','b']
}

#df1 = pd.DataFrame(data, index = ["day1", "day2", "day3"])
df1 = pd.DataFrame(data)
df2 = pd.DataFrame(data2)
print(df1) 
print(df2)

   calories  duration        fat
0       420        50      tired
1       380        40     hungry
2       390        45  Energetic
   x  y
0  1  a
1  2  b


In [113]:
import pandas as pd

#df = pd.read_csv('data.csv')

#print(df.to_string()) 

#pd.options.display.max_rows = 9999

print(pd.options.display.max_rows) 

60


In [114]:
df1.head()

,calories,duration,fat
0,420,50,tired
1,380,40,hungry
2,390,45,Energetic


# learn how to read csv into dataframe

In [115]:
from langchain_experimental.tools import PythonAstREPLTool
from langchain_openai import AzureChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd
import os
from langchain_core.prompts import format_document
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import streamlit as st
from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import ChatPromptTemplate

llm = AzureChatOpenAI(
            deployment_name = "gpt-35-turbo-16k",
            azure_endpoint= st.secrets['OPENAI_API_ENDPOINT'],
            openai_api_type="azure",
            openai_api_version = "2023-07-01-preview"
        )

In [116]:
# run some pandas operations commands (sum, max etc)

In [117]:
#Chain
ai_msg = llm.invoke(
    "I have a pandas DataFrame 'df1' with columns 'calories' and 'duration'. Write code to compute the correlation between the two columns. Return Markdown for a Python code snippet and nothing else."
)
print(ai_msg.content)

correlation = df1['calories'].corr(df1['duration'])
correlation



```python
correlation = df1['calories'].corr(df1['duration'])
```


0.960768922830523

In [118]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

tool = PythonAstREPLTool(locals={"df": df})


In [97]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
llm_with_tools.invoke(
    "I have a dataframe 'df1' and want to know the correlation between the 'calories' and 'duration' columns"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hobvsLrgrRBQt8Qrpig7bg0B', 'function': {'arguments': '{\n  "query": "import pandas as pd\\ndf1 = pd.DataFrame({\'calories\': [120, 150, 100, 200], \'duration\': [10, 15, 8, 12]})\\ncorrelation = df1[\'calories\'].corr(df1[\'duration\'])\\ncorrelation"\n}', 'name': 'python_repl_ast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 123, 'total_tokens': 195}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-85db8e73-549d-49e5-a877-a803cbe67a5e-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'q

In [100]:
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)
(llm_with_tools | parser).invoke(
    "I have a dataframe 'df1' and want to know the correlation between the 'calories' and 'duration' columns"
)

{'query': "df1['calories'].corr(df1['duration'])"}

In [101]:
df1['calories'].corr(df1['duration'])


0.960768922830523

In [102]:
system = f"""You have access to a pandas dataframe `df`. \
Here is the output of `df.head().to_markdown()`:
{df1.head().to_markdown()}
Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
code_chain = prompt | llm_with_tools | parser
code_chain.invoke({"question": "What's the correlation between 'calories' and 'duration'"})

{'query': "df['calories'].corr(df['duration'])"}

In [103]:
chain = prompt | llm_with_tools | parser | tool  # noqa
chain.invoke({"question": "What's the correlation between 'calories' and 'duration'"})

0.960768922830523

In [51]:
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser


chain.invoke(
    "I have a dataframe 'df1' and want to know the correlation between the 'calories' and 'duration' columns"
)

[{'args': {'query': "import pandas as pd\n\ndf1 = pd.DataFrame({'calories': [100, 200, 300, 400], 'duration': [10, 20, 30, 40]})\n\ncorrelation = df1['calories'].corr(df1['duration'])\ncorrelation"},
  'type': 'python_repl_ast'}]

In [52]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe `df1`. \
Here is the output of `df1.head().to_markdown()`: 
Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas.
Respond directly to the question once you have enough information to answer it."""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]




In [104]:
chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser | tool)
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["tool_output", "response"])
)

chain.invoke({"question": "What's the correlation between 'calories' and 'duration'"})

{'tool_output': 0.960768922830523,
 'response': "df['calories'].corr(df['duration'])"}

## Agents AVenger

In [120]:
from langchain_experimental.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm, df1, agent_type="openai-tools", verbose=True)
agent.invoke(
    {
        "input": "What's the correlation between 'calories' and 'duration'? is that greater than the correlation between 'calories' and 'fat'?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[['calories', 'duration']].corr().iloc[0,1]"}`


0.9607689228305226
Invoking: `python_repl_ast` with `{'query': "df[['calories', 'fat']].corr().iloc[0,1]"}`


ValueError: could not convert string to float: 'tired'The correlation between 'calories' and 'duration' is approximately 0.961. However, we cannot calculate the correlation between 'calories' and 'fat' because 'fat' is a categorical variable.

> Finished chain.


{'input': "What's the correlation between 'calories' and 'duration'? is that greater than the correlation between 'calories' and 'fat'?",
 'output': "The correlation between 'calories' and 'duration' is approximately 0.961. However, we cannot calculate the correlation between 'calories' and 'fat' because 'fat' is a categorical variable."}

# try to run this code

In [125]:
df1 , df2


(   calories  duration        fat
 0       420        50      tired
 1       380        40     hungry
 2       390        45  Energetic,
    x  y
 0  1  a
 1  2  b)

In [129]:
df1['Name'] = ['esfarhan','goo','jingwen']

In [131]:
df1

,calories,duration,fat,Name
0,420,50,tired,esfarhan
1,380,40,hungry,goo
2,390,45,Energetic,jingwen


In [145]:
df3 = pd.DataFrame()
df3["resigned"] = [True, False, True]
df3

,resigned
0,True
1,False
2,True


In [146]:


df_1 = df1
df_2 = df2
df_3 = df3

tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2, "df_3":df_3})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """```python
{df_name}.head().to_markdown()
>>> {df_head}
```"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df_1, "df_1"), (df_2, "df_2"),(df_3, "df_3")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

chain = prompt | llm_with_tool | parser | tool
chain.invoke(
    {
        "question": "Calculate the difference between calories and x. Then return the name with the highest difference and the condition of the its resigned column."
        #"question": "Calculate the difference between x and calories ?"
    }
)

('esfarhan', True)

In [60]:


from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser




tool = PythonAstREPLTool(locals={"df_1": df1, "df_2": df2})
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

(llm_with_tools | parser).invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Age' and 'Fare' columns"
)
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """```python
{df_name}.head().to_markdown()
>>> {df_head}
```"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df1, "df_1"), (df2, "df_2")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

chain = prompt | llm_with_tool | parser | tool
chain.invoke(
    {
        "question": "return the difference in the correlation between age and fare and the correlation between fare and survival"
    }
)

ValidationError: 1 validation error for PythonInputs
query
  field required (type=value_error.missing)

In [19]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Using LangSmith is recommended but not required. Uncomment below lines to use.
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

#!wget https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv -O titanic.csv

import pandas as pd

df = pd.read_csv("titanic.csv")
print(df.shape)
print(df.columns.tolist())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd4 in position 12: invalid continuation byte

In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic", engine, index=False)

db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM titanic WHERE Age < 2;")

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI


# AGENT SQL
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)


In [ ]:
agent_executor.invoke({"input": "what's the average age of survivors"})